In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AI+X 중급

/content/drive/MyDrive/Colab Notebooks/AI+X 중급


In [ ]:
! pip install gradio
! pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
import openai
import gradio as gr
import json
from datetime import date

# 저장해둔 api key 불러옴
key_file = open("chatgpt_api_key.txt", "r")
key_data = key_file.read()
key_file.close()  

# 발급받은 API 키 설정
OPENAI_API_KEY = key_data

# openai API 키 인증
openai.api_key = OPENAI_API_KEY

# 저장해둔 dzm의 role 불러옴
dzm_file = open("DZM.txt", "r")
dzm_data = dzm_file.read()
dzm_file.close()  

user_name_list = []

# 직접 날짜 받아올때 사용
# date = date.today()
# today = date.isoformat()
# today
today = "2023-06-08"
today

# 나중에 user_name은 웹에서 받아오기
user_name = "Youjin"
user_name_list.append(user_name)
file_name = user_name + "_history.json"

# message_history
message_history = []

if user_name in user_name_list:
  with open(file_name, 'r') as f:   
    message_history = json.load(f)

message_history.append({"role": "system", "content": f"{dzm_data}"+" And my name is "+f"{user_name}"})
message_history.append({"role": "system", "content": "Today is " + f"{today}." + " Please remember I worked out today. The days I didn't say were the days I didn't exercise."})
init_len = len(message_history)

# chatbot 구현
def chat_model(input):

    message_history.append({"role": "user", "content": f"{input}"})

    chat = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo", 
            messages = message_history,
        )

    reply_content = chat.choices[0].message.content
    message_history.append({"role": "assistant", "content": f"{reply_content}"})

    with open(file_name, 'w') as outfile:
      json.dump(message_history, outfile)

    response = [(message_history[i]["content"], message_history[i+1]["content"]) for i in range(init_len, len(message_history)-1, 2)]
    
    return response

# gradio
with gr.Blocks() as demo: 

    chatbot = gr.Chatbot() 

    with gr.Row(): 
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

    txt.submit(chat_model, txt, chatbot)
 
    txt.submit(None, None, txt, _js="() => {''}") 

gr.Interface(fn=chat_model, inputs=txt, outputs=chatbot, title="DZM",
             description="How was your workout today?",
             theme="compact").launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:666: UserWarning: Cannot load compact. Caught Exception: The space compact does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2b72d9117624604e69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
